In [1]:
from PattRecClasses import HMM_TA 
import numpy as np
import scipy.stats


In [2]:
'''
Multivariate Gaussian Distribution Class
'''
class multigaussD:
    mean = np.array([0])
    cov = np.array([[0]])
    def __init__(self, mu, C):
        if C.shape[0] is not C.shape[1]:
            print("error, non-square covariance matrix supplied")
            return
        if mu.shape[0] is not C.shape[0]:
            print("error, mismatched mean vector and covariance matrix dimensions")
            return
        self.mean = mu
        if np.where(np.diag(C)==0)[0].shape[0] != 0:
            C += np.diagflat(np.ones(C.shape[0])/10000)
        C[np.isnan(C)]=1
        self.cov = C
        return
    def random(self, num):
        return np.random.multivariate_normal(self.mean, self.cov, num)
    def rand(self):
        return np.random.multivariate_normal(self.mean, self.cov, 1)[0]
    def likelihood(self, X):
        p = scipy.stats.multivariate_normal(self.mean, self.cov, 1)
        pd = p.pdf(X)
        return pd
    def loghood(self, X):
        return np.log(self.likelihood(X))
    def getmean(self):
        return self.mean
    def getcov(self):
        return self.cov
    
def prob(x, B):
    T = x.shape[0]
    N = B.shape[0]
    res = np.zeros((T, N))
    for i in range(T):
        for j in range(N):
            res[i,j] = B[j].likelihood(x[i])
    scaled = np.zeros(res.shape)
    for i in range(scaled.shape[0]):
        for j in range(scaled.shape[1]):
            scaled[i, j] = res[i,j]/np.amax(res[i])
    return res, scaled


def logprob(x, B):
    res, scaled = prob(x,B)
    return np.log(res), np.log(scaled)
   

In [3]:
 
'''
TESTCASES
'''

# Define a HMM
q = np.array([0.8, 0.2])
A = np.array([[0.95, 0.05],
              [0.30, 0.70]])

means = np.array( [[0, 0], [2, 2]] )
covs  = np.array( [[[1, 2],[2, 4]], 
                   [[1, 0],[0, 3]]] )

B = np.array([multigaussD(means[0], covs[0]),
              multigaussD(means[1], covs[1])])


hm  = HMM_TA.HMM(q, A, B)
obs = np.array([ hm.rand(100)[0] for _ in range(10) ])

print('True HMM parameters:')
print('q:')
print(q)
print('A:')
print(A)
print('B: means, covariances')
print(means)
print(covs)

# Estimate the HMM parameters from the obseved samples
# Start by. assigning initial HMM parameter values,
# then refine these iteratively
qstar = np.array([0.8, 0.2])
Astar = np.array([[0.5, 0.5], [0.5, 0.5]])

meansstar = np.array( [[0, 0], [0, 0]] )

covsstar  = np.array( [[[1, 0],[0, 1]], 
                       [[1, 0],[0,1]]] )

Bstar = np.array([multigaussD(meansstar[0], covsstar[0]),
                  multigaussD(meansstar[1], covsstar[1])])


hm_learn = HMM_TA.HMM(qstar, Astar, Bstar)

print("Running the Baum Welch Algorithm...")
hm_learn.baum_welch(obs, 20, prin=1, uselog=False)

# Test the Viterbi algorithm
print("Running the Viterbi Algorithm...")
obs, true_states = hm.rand(100)

print("True States:\n",true_states)
print("Predicted States:\n", hm_learn.viterbi(obs))

True HMM parameters:
q:
[0.8 0.2]
A:
[[0.95 0.05]
 [0.3  0.7 ]]
B: means, covariances
[[0 0]
 [2 2]]
[[[1 2]
  [2 4]]

 [[1 0]
  [0 3]]]
Running the Baum Welch Algorithm...
Estimated a:
[0.79929056 0.20070944]

Estimated A:
[[0.95315598 0.04684402]
 [0.36568958 0.63431042]]

Estimated means:
[[0.06704437 0.13416694]
 [2.04734672 2.32102668]]

Estimated covariances:
[[[1.00807818 2.01624535]
  [2.01624535 4.03267526]]

 [[1.0888189  0.20522544]
  [0.20522544 3.22073201]]]
Running the Viterbi Algorithm...
True States:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Predicted States:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:310: RuntimeWarning: divide by zero encountered in log
  return np.log(res), np.log(scaled)
